In [52]:
import json
import datetime
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [53]:
file1=open('playerStats.json')
file2=open('teamsData.json')
playerStats=json.load(file1)
teamsData=json.load(file2)

/tmp/ipykernel_25863/2565280006.py:1: ResourceWarning: unclosed file <_io.TextIOWrapper name='playerStats.json' mode='r' encoding='UTF-8'>
  file1=open('playerStats.json')
/tmp/ipykernel_25863/2565280006.py:2: ResourceWarning: unclosed file <_io.TextIOWrapper name='teamsData.json' mode='r' encoding='UTF-8'>
  file2=open('teamsData.json')


# nationality analysis
---
- excat same analysis can be done for `bowlingStyle`[multiple category] 
- or for `right/left` handbad,bowl [binary]
- a bit diff analysis for age is done below

In [54]:
def getNations(playerStats):
    nationList=[]
    for player in playerStats:
        nation=player['player']['nationality']
        if not (nation in nationList):
            nationList.append(nation)
    return nationList

nations=getNations(playerStats)
# print(nations)


In [55]:
# in these fucntions other info lists also can be added like APIs
def getTeamNations(id,playerStats=playerStats,nationNames=nations):
    #set up dictoinary
    teamNations={'teamID':id}
    for nation in nationNames:
        teamNations.setdefault(nation,0)
    #add an extra key
    teamNations.setdefault('Foreigner',0)

    for player in playerStats:
        if player['teamID']==id:
            nation=player['player']['nationality']
            if nation=='Indian':
                teamNations[nation]+=1
            else:
                teamNations['Foreigner']+=1
                teamNations[nation]+=1
    
    return teamNations


In [56]:
nationalityList=[]
for team in teamsData:
    id=team['id']
    data=getTeamNations(id)
    data['Team Name']=team['abbreviation']
    nationalityList.append(data)

nationalityDF=pd.DataFrame(nationalityList)
nationalityDF.to_csv('nationlityOfTeams.csv',index=False)

In [57]:
totalOverall=list(nationalityDF.iloc[:,1:-1].sum(axis=0))
nationLabels=list(nationalityDF.columns)[1:-1]
overallDf=pd.DataFrame({'Nation':nationLabels,'Player Count':totalOverall})
overallDf.sort_values('Player Count',axis=0,inplace=True)

In [58]:
plt.barh(y=overallDf.Nation,width=overallDf['Player Count'],color='#FDB913')
plt.ylabel('Nationalities')
plt.xlabel('player count of particular nation')
plt.tight_layout()
plt.savefig('plots/nationality.png')
plt.show()

In [59]:
ageData =list()

for player in playerStats:
    data=player['player']
    birthday=data['dateOfBirth']
    #convert into date time object
    birthday=(datetime.datetime.strptime(birthday,'%Y-%m-%d').date())
    today=datetime.date.today()
    daysOld=(today-birthday).days
    daysInYear=365.25
    age=int(daysOld//daysInYear)

    ageData.append({'id':data['id'],'teamID':player['teamID'],'name':data['fullName'],'age':age})

In [60]:
ageDF=pd.DataFrame(ageData)
ageDF.sort_values('id',axis=0,inplace=True)

In [61]:
# basic sumarry of age data
print('mean and median age in tournament >>> ',(round(ageDF.age.describe()['mean'],2),ageDF.age.describe()['50%']))

mean and median age in tournament >>>  (28.32, 28.0)


In [62]:
teamNames=[team['abbreviation'] for team in teamsData]
ageMeans=list(ageDF[['age','teamID']].groupby('teamID').mean()['age'])
plt.barh(y=teamNames,width=ageMeans,color='orange')
plt.xlabel('team avg age')
plt.ylabel('team abbr')
plt.tight_layout()
plt.savefig('plots/age.png')
plt.show()

# getCSV Data for player,bowl,bat,field

In [63]:
playerList=[]
bowlingstatsList=[]
battingstatsList=[]
fieldingstatsList=[]
for player in playerStats:
    item=player['player']
    item['teamID']=player['teamID']
    playerList.append(item)
    playerID=item['id'] # <-- add player id to identify in analysis
    if len(player['stats'])>0:
        # getting iplt20 data
        for data in player['stats']:
            if data['matchType']=='IPLT20': #<-- chossing IPLT20 data from 4 different stats
                statData=data
                bowlstat=statData['bowlingStats']
                bowlstat['teamID']=player['teamID']
                bowlstat['playerID']=playerID
                bowlingstatsList.append(bowlstat)
                batstat=statData['battingStats']
                batstat['teamID']=player['teamID']
                batstat['playerID']=playerID
                battingstatsList.append(batstat)
                fieldstat=statData['fieldingStats']
                fieldstat['teamID']=player['teamID']
                fieldstat['playerID']=playerID
                fieldingstatsList.append(fieldstat)

In [64]:
playerDF=pd.DataFrame(playerList)
bowlDF=pd.DataFrame(bowlingstatsList)
batDF=pd.DataFrame(battingstatsList)
fieldDF=pd.DataFrame(fieldingstatsList)

playerDF.to_csv('datasets/playerdata.csv',index=False)
bowlDF.to_csv('datasets/bowlingdata.csv',index=False)
batDF.to_csv('datasets/battingdata.csv',index=False)
fieldDF.to_csv('datasets/fieldingdata.csv',index=False)